
#Ejemplo TPU
En este documento se evidencia la forma en la que se replica el ejemplo presentado por Sotaquirá en su video introductorio a Google Colab. Este cuaderno corresponde al caso en el que se usa la TPU para programar el modelo.

La preparación de datos y especificación es esencialmente igual que en el caso para CPU. Sin embargo es necesario, primero, establecer el entorno de ejecución en TPU.

In [1]:
import numpy as np
import os
import gzip
import tensorflow as tf

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])  # Detectar TPU
  print('TPU encontrada ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: TPU no encontrada!')


TPU encontrada  ['10.89.148.162:8470']


#Activando la TPU

In [2]:
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.89.148.162:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.89.148.162:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [3]:
def load_mnist(ruta, tipo='train'):

    ruta_categorias = os.path.join(ruta, '%s-labels-idx1-ubyte.gz' % tipo) 
    ruta_imagenes = os.path.join(ruta, '%s-images-idx3-ubyte.gz' % tipo)

    with gzip.open(ruta_categorias, 'rb') as rut_cat:
     etiquetas = np.frombuffer(rut_cat.read(), dtype=np.uint8, offset=8)

    with gzip.open(ruta_imagenes, 'rb') as rut_imgs:
     imagenes = np.frombuffer(rut_imgs.read(), dtype=np.uint8, offset=16).reshape(len(etiquetas),784)

    return imagenes, etiquetas



#Accediendo a Google Drive

In [18]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

ruta='gdrive/My Drive/Colab Notebooks/Tarea 1/'

X_train, Y_train = load_mnist(ruta, tipo='train')
X_test, Y_test = load_mnist(ruta, tipo='test')

Mounted at /content/gdrive


#Ajustando los datos

In [19]:
X_train = X_train[0:59904,:]
X_test = X_test[0:9984,:]
Y_train = Y_train[0:59904]
Y_test = Y_test[0:9984]

X_train = np.reshape(X_train,(59904,28,28,1))
X_test = np.reshape(X_test,(9984,28,28,1))

#Implementando el modelo

In [6]:
%tensorflow_version 2.x
print('Versión de TensorFlow: ' +tf.__version__)

tf.random.set_seed(200)

Versión de TensorFlow: 2.8.0


# Creación del modelo

In [20]:
with tpu_strategy.scope():
  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
  model.add(tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='elu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
  model.add(tf.keras.layers.Dropout(0.25))

  model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
  model.add(tf.keras.layers.Conv2D(128, (5, 5), padding='same', activation='elu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
  model.add(tf.keras.layers.Dropout(0.25))

  model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
  model.add(tf.keras.layers.Conv2D(256, (5, 5), padding='same', activation='elu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
  model.add(tf.keras.layers.Dropout(0.25))

  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(256))
  model.add(tf.keras.layers.Activation('elu'))
  model.add(tf.keras.layers.Dropout(0.5))
  model.add(tf.keras.layers.Dense(10))
  model.add(tf.keras.layers.Activation('softmax'))
  model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_3 (Batc  (None, 28, 28, 1)        4         
 hNormalization)                                                 
                                                                 
 conv2d_3 (Conv2D)           (None, 28, 28, 64)        1664      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 14, 14, 64)       0         
 2D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 14, 14, 64)        0         
                                                                 
 batch_normalization_4 (Batc  (None, 14, 14, 64)       256       
 hNormalization)                                                 
                                                      

In [14]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

#Generando el Checkpoint
Para el caso de TPU, la forma de guardar checkpoints es usando Google Cloud Service, por lo tanto, para esta tarea, no se guarda checkpoint de este modelo 

In [17]:
import timeit

x_train = X_train.astype(np.float32)
y_train = Y_train.astype(np.float32)
x_test = X_test.astype(np.float32)
y_test = Y_test.astype(np.float32)

def entrenamiento_tpu():
  model.fit(x_train,y_train,validation_data=(x_test,y_test), batch_size=128, epochs=2, verbose=1)
  
  return None

tpu_time = timeit.timeit('entrenamiento_tpu()', number=1, setup='from __main__ import entrenamiento_tpu')

Epoch 1/2
468/468 [==============================] - 10s 21ms/step - loss: 0.2654 - accuracy: 0.9024 - val_loss: 0.2610 - val_accuracy: 0.9074
Epoch 2/2
468/468 [==============================] - 10s 22ms/step - loss: 0.2422 - accuracy: 0.9112 - val_loss: 0.2809 - val_accuracy: 0.9045
